# `nepare` ASAP `polaris` Competition

This notebook demonstrates using Neural Pairwise Regression (via `nepare`) with the `polaris` benchmarking library to compete in the ASAP discovery competition.

## Requirements
Python 3.10+ (originally run on 3.12)
 - polaris-lib
 - pandas
 - rdkit
 - lightning
 - torch
 - fastprop
 - chemprop 2.1
 - ipywidgets

You will also need to run `pip install .` in the repository's root directory to install `nepare`.

## `polaris` Setup

After running `polaris login` on the command line, we can import everything (checking that the version is recent enough) and then download the benchmark data.

In [10]:
import polaris as po
import pandas as pd

In [11]:
%%capture
competition = po.load_competition("asap-discovery/antiviral-potency-2025")
# or
# competition = po.load_competition("asap-discovery/antiviral-admet-2025")

Choose the type of embedding to use - either a fixed descriptor-based embedding with mordred (great with limited data) or a learnable embedding with ChemProp (more general representation).
The choice as shown is configured based on previous experimentation.

In [12]:
_m, _c, _b = "mordred", "chemprop", "combined"
EMBEDDING = {
    'HLM': _c,
    'MLM': _b,
    'KSOL': _m,
    'LogD': _m,
    'MDR1-MDCKII': _m,
    'pIC50 (MERS-CoV Mpro)': _b,
    'pIC50 (SARS-CoV-2 Mpro)': _b,
}
# could consider a combined ebedding where the dataset returns two tuples (0 is chemprop features, 1 is mordred) and then a slightly modified learnable_ebedding module that does what the current chemprop one does except conatenates the output with mordred

In [13]:
train, test = competition.get_train_test_split()
test_df: pd.DataFrame = test.as_dataframe()
train_df: pd.DataFrame = train.as_dataframe()

In [ ]:
train_df

## Train Per-Task Models

In [15]:
from pathlib import Path

import torch
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from chemprop.nn.agg import MeanAggregation
from chemprop.nn.message_passing import BondMessagePassing
from fastprop.data import standard_scale, inverse_standard_scale

from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.nn import LearnedEmbeddingNeuralPairwiseRegressor, NeuralPairwiseRegressor
from nepare.inference import predict

from utils.splitting import split
from utils.chemprop_wrappers import smiles2molgraphcache, collate_fn, ChemPropEmbedder
from utils.mordred_wrappers import smi2features
from utils.metrics import evaluate_predictions
from utils.combined_embeddings import MordredChemPropEmbedder, collate_fn_combined

In [ ]:
predictions = {}
val_score = []
output_dir = Path("lightning_logs")
tasks = list(competition.target_cols)
for task_n, task_name in enumerate(tasks):
    dataset_kwargs = dict(batch_size=64)
    task_df = train_df[["CXSMILES", task_name]].copy()
    task_df.dropna(inplace=True)
    task_df.reset_index(inplace=True)
    train_idxs, val_idxs = split(task_df["CXSMILES"], test_df["CXSMILES"])
    train_targets = torch.tensor(task_df[task_name].iloc[train_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    train_targets, target_means, target_vars = standard_scale(train_targets)
    val_targets = torch.tensor(task_df[task_name].iloc[val_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    val_targets = standard_scale(val_targets, target_means, target_vars)
    if EMBEDDING[task_name] == "mordred":
        Model = NeuralPairwiseRegressor
        train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
        val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
        test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
        # setup the datasets
        train_dataset = PairwiseAugmentedDataset(train_features, train_targets, how='full')
        val_dataset = PairwiseAnchoredDataset(train_features, train_targets, val_features, val_targets, how='full')
        val_absolute_dataset = PairwiseInferenceDataset(train_features, train_targets, val_features, how='full')
        test_dataset = PairwiseInferenceDataset(train_features, train_targets, test_features, how='full')
        # build the model
        npr = Model(train_features.shape[1], 70, 3)
    elif EMBEDDING[task_name] == "chemprop":
        Model = LearnedEmbeddingNeuralPairwiseRegressor
        # featurize the data
        train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
        val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
        test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
        # setup the datasets
        dataset_kwargs["collate_fn"] = collate_fn
        train_dataset = PairwiseAugmentedDataset(train_mgc, train_targets, how='sut')
        val_dataset = PairwiseAnchoredDataset(train_mgc, train_targets, val_mgc, val_targets, how='half')
        val_absolute_dataset = PairwiseInferenceDataset(train_mgc, train_targets, val_mgc, how='half')
        test_dataset = PairwiseInferenceDataset(train_mgc, train_targets, test_mgc, how='half')
        # build the model
        mp = BondMessagePassing(d_h=400)
        agg = MeanAggregation()
        embedder = ChemPropEmbedder(mp, agg)
        npr = Model(embedder, 400, 200, 2, lr=5e-5)
    elif EMBEDDING[task_name] == "combined":
        Model = LearnedEmbeddingNeuralPairwiseRegressor
        # featurize the data
        train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
        val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
        test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
        train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
        val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
        test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
        train_tuples = [(train_mgc[i], train_features[i, :]) for i in range(len(train_targets))]
        val_tuples = [(val_mgc[i], val_features[i, :]) for i in range(len(val_targets))]
        test_tuples = [(test_mgc[i], test_features[i, :]) for i in range(len(test_mgc))]
        # setup the datasets
        train_dataset = PairwiseAugmentedDataset(train_tuples, train_targets, how='full')
        val_dataset = PairwiseAnchoredDataset(train_tuples, train_targets, val_tuples, val_targets, how='full')
        val_absolute_dataset = PairwiseInferenceDataset(train_tuples, train_targets, val_tuples, how='full')
        test_dataset = PairwiseInferenceDataset(train_tuples, train_targets, test_tuples, how='full')
        dataset_kwargs["collate_fn"] = collate_fn_combined
        # build the model
        mp = BondMessagePassing(d_h=1_000, depth=2)
        agg = MeanAggregation()
        embedder = MordredChemPropEmbedder(mp, agg)
        npr = Model(embedder, mp.output_dim + train_features.shape[1], 1_000, 2, lr=5e-5)


    # classic lightning training, inference
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **dataset_kwargs)
    val_loader = torch.utils.data.DataLoader(val_dataset, **dataset_kwargs)
    val_absolute_loader = torch.utils.data.DataLoader(val_absolute_dataset, **dataset_kwargs)
    predict_loader = torch.utils.data.DataLoader(test_dataset, **dataset_kwargs)
    early_stopping = EarlyStopping(monitor="validation/loss", patience=3)
    name = "_".join([EMBEDDING[task_name], task_name])
    model_checkpoint = ModelCheckpoint(dirpath=output_dir / name, monitor="validation/loss")
    logger = TensorBoardLogger(save_dir=output_dir, name=name, default_hp_metric=False)
    trainer = lightning.Trainer(max_epochs=50, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint], logger=logger)
    trainer.fit(npr, train_loader, val_loader)
    npr = Model.load_from_checkpoint(model_checkpoint.best_model_path)
    y_pred, y_stdev = predict(npr, val_absolute_loader)
    # leave in the scaled space so that the average is weighted equally among the targets
    results_dict = evaluate_predictions(val_targets.flatten().numpy(), y_pred.flatten().numpy())
    val_score.append((len(val_targets), results_dict["MAE"]))
    # now go back to correct scale
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    y_true = inverse_standard_scale(val_targets, target_means, target_vars)

    # metric reporting, for our own information
    print(f"{task_name=}:\n - {"\n - ".join([f'{name}: {score:.4f}' for name, score in evaluate_predictions(y_true.flatten().numpy(), y_pred.flatten().numpy()).items()])}")

    # actual predictions
    y_pred, y_stdev = predict(npr, predict_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    predictions[task_name] = y_pred.flatten().tolist()

In [ ]:
print(f"Validation Scaled Weighted Average MAE {sum(n*s for n, s in val_score) / sum(n for n, _ in val_score):.4f}")

This last block is commented out because it will fail (unless you are me) - you can replace the inputs with your own if you are submitting this for yourself.

In [18]:
# competition.submit_predictions(
#     predictions=predictions,
#     prediction_name="nepare",
#     prediction_owner="jacksonburns",
#     report_url="https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/meta",
#     github_url = "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_asap/main.ipynb",
#     description = "Neural Pairwise Regression with ChemProp as a learnable embedding",
# )